In [1]:
import numpy as np
import pandas as pd

## Back-Propagation in Multi-Neuron 1 layer network

In [86]:
def nn_generalized_multi(x,y,wh,wo,lr,n_epoch):
    y_h = np.dot(x,wh)
    y_hat = np.dot(y_h,wo)
    for epoch in range(n_epoch):
        dcostdwo = 2*np.dot(-y_h.T,(y-y_hat))
        dcostdwh = 2*np.dot(np.dot(-x.T,(y-y_hat)),wo.T)
        wo = wo - lr*dcostdwo
        wh = wh - lr*dcostdwh
        y_h = np.dot(x,wh)
        y_hat = np.dot(y_h,wo)
        cost = np.sum((y-y_hat)**2)
        if epoch%1000==0:
            print("epoch :{:d} cost:{:f}".format(epoch,cost))
        if cost <= 0.001:
            print("epoch :{:d} cost:{:f}".format(epoch,cost))
            break
    return wh,wo

In [87]:
x = np.array([[3,8,10],
             [10,7,2],
             [2,9,15],
             [1,12,34],
             [3,1,9]])
y = [[51],[60],[57],[85],[22]]
h_size = 5
o_size = 1
wh = np.random.random((x.shape[1],h_size))
wo = np.random.random((wh.shape[1],o_size))
lr = 0.00001
n_epoch=10000
wh,wo = nn_generalized_multi(x,y,wh,wo,lr,n_epoch)
print "y_hat_final:\n{:s}".format(np.dot(np.dot(x,wh),wo))


epoch :0 cost:3081.635879
epoch :1000 cost:3.361895
epoch :2000 cost:0.156998
epoch :3000 cost:0.006833
epoch :3611 cost:0.000997
y_hat_final:
[[ 50.98748244]
 [ 60.00745208]
 [ 56.9873442 ]
 [ 85.00328881]
 [ 22.02478131]]


## Decouple forward and backward propagation

In [75]:
def create_network(x,y,hidden_layer_size,output_layer_size):
    weight_h = np.random.random((x.shape[1],hidden_layer_size))
    weight_o = np.random.random((weight_h.shape[1],output_layer_size))
    return weight_h,weight_o 

def forward_propagation(x,y,weight_h,weight_o):
    y_h = np.dot(x,weight_h)
    y_hat = np.dot(y_h,weight_o)
    cost = 0.5*np.sum((y-y_hat)**2)
    return y_h,y_hat,cost

def backward_propagation(x,y,weight_h,weight_o,y_h,y_hat):
    dcostdwo = np.dot(-y_h.T,(y-y_hat))
    dcostdwh = np.dot(np.dot(-x.T,(y-y_hat)),weight_o.T)
    weight_o = weight_o - lr*dcostdwo
    weight_h = weight_h - lr*dcostdwh
    return weight_h,weight_o

def train_network(x,y,weight_h,weight_o,lr,n_epoch):
    cost_list = []
    for epoch in range(n_epoch):
        y_h,y_hat,cost = forward_propagation(x,y,weight_h,weight_o)

        if epoch%1000==0:
            print("epoch :{:d} cost:{:f}".format(epoch,cost))

        if cost <= 0.001:
            print("epoch :{:d} cost:{:f}".format(epoch,cost))
            break

        weight_h,weight_o = backward_propagation(x,y,weight_h,weight_o,y_h,y_hat)
        cost_list.append([epoch,cost])
        cost_list = pd.DataFrame(cost_list,columns=['epoch','cost'])
    return weight_h,weight_o,cost_list

def predict(x,weight_h,weight_o):
    y_h = np.dot(x,weight_h)
    y_hat = np.dot(y_h,weight_o)
    return y_hat

In [88]:
x = np.array([[3,8,10],
             [10,7,2],
             [2,9,15],
             [1,12,34],
             [3,1,9]])
y = [[51],[60],[57],[85],[22]]

hidden_layer_size = 2
output_layer_size = 1

weight_h,weight_o  = create_network(x,y,hidden_layer_size,output_layer_size) # Define network
weight_h,weight_o,cost_list = train_network(x,y,weight_h,weight_o,lr,n_epoch) # train network
y_pred = predict(x,weight_h,weight_o)    # predict
print "y_hat_final:\n{:s}".format(y_pred)

epoch :0 cost:5340.079448
epoch :1000 cost:0.002987
epoch :1673 cost:0.000999
y_hat_final:
[[ 51.01757006]
 [ 59.98918462]
 [ 57.01777865]
 [ 84.99522464]
 [ 21.96487901]]


In [201]:
layers = [2,2,1]
def create_network(x,y,layers):
    layers = [x.shape[1]] + layers
    layers_weights = []
    for layer in range(len(layers)-1):
        weight = np.random.random((layers[layer],layers[layer+1]))
        layers_weights.append(weight)
    return layers_weights

weights = create_network(x,y,layers)

In [207]:
def forward_propagation(x,y,weights):
    y_h = x
    y_hats = []
    for weight in weights:
        y_h = np.dot(y_h,weight)
        y_hats.append(y_h)
    cost = 0.5*np.sum((y-y_hats[-1])**2)
    return y_hats,cost

y_hats,cost = forward_propagation(x,y,weights)

In [205]:

for weight in weights:
    print weight.shape

(3L, 2L)
(2L, 2L)
(2L, 1L)


In [208]:
y_hats = [x] + y_hats
for y_hat in y_hats:
    print y_hat.shape

(5L, 3L)
(5L, 2L)
(5L, 2L)
(5L, 1L)


In [210]:
y_hats_reversed = list(reversed(y_hats))
weights_reversed = list(reversed(weights))
for w in weights_reversed:
    print w.shape
print "/n"
for y in y_hats_reversed:
    print y.shape

(2L, 1L)
(2L, 2L)
(3L, 2L)
/n
(5L, 1L)
(5L, 2L)
(5L, 2L)
(5L, 3L)


In [213]:
def backward_propagation(x,y,weights,y_hats):
    y_hats = [x] + y_hats
    y_hats_reversed = list(reversed(y_hats))
    weights_reversed = list(reversed(weights))
    dcostdw_list = []
    
    dcostdw = np.dot(-y_hats_reversed[1].T,(y-y_hats_reversed[0]))
    dcostdw_list.append(dcostdw)
    weights_reversed[0] = weights_reversed[0] - dcostdw
    for y_hat_index in range(1,len(y_hats_reversed)):
        print y_hat_index
        dcostdw = np.dot(np.dot(-y_hats_reversed[y_hat_index+1].T,(y-y_hats_reversed[0])),weights_reversed[y_hat_index+1].T)
        dcostdw_list.append(dcostdw)
        #weights_reversed[y_hat_index] = weights_reversed[y_hat_index] - lr*dcostdw
#     dcostdwo = np.dot(-y_h.T,(y-y_hat))
#     dcostdwh = np.dot(np.dot(-x.T,(y-y_hat)),weight_o.T)
#     weight_o = weight_o - lr*dcostdwo
#     weight_h = weight_h - lr*dcostdwh
    weights = reversed(weights)
    return weights

In [214]:
backward_propagation(x,y,weights,y_hats)

1


ValueError: shapes (2,3) and (2,3) not aligned: 3 (dim 1) != 2 (dim 0)